In [1]:
from chase.base import *
from chase.utils import *
from chase.fit import *
from mypy.explib.frey2014 import frey2014
pd.set_option('display.max_colwidth', 100)

In [2]:
data = pd.read_csv('data/frey_data.csv', index_col=0)
data.head(10)

,subject,problem,group,choice,samplesize
0,1,S1G1,0,0,12
1,1,S2G1,0,1,10
2,1,S3G1,0,0,10
3,1,S4G1,0,0,10
4,1,S5G1,0,0,10
5,1,S6G1,0,1,10
6,1,S7G1,0,0,10
7,1,S8G1,0,0,12
8,1,S9G1,0,0,10
9,1,S10G1,0,0,10


In [4]:
problems = {gid: frey2014.get_options(gid) for gid in data.problem.unique()}

In [5]:
switchfreq = frey2014.data.groupby('partid').apply(lambda x: np.mean((x['switchcount'])/x['samplesize']))
print switchfreq[30]

0.0627200491398


In [6]:
emp_p_switch = {}
for sid in frey2014.subjects():
    arr = []
    for a in frey2014.streak_lengths_by_subject(sid).values:
        for x in a:
            arr.append(x)
    emp_p_switch[sid] = np.round(1 / np.mean(arr), 3)

In [7]:
import pickle

In [8]:
with open('data/frey_switchrates.pkl', 'w') as fp:
    pickle.dump(emp_p_switch, fp)

# Fit individual subjects

In [8]:
# a list of possible free parameters and their ranges
# and starting point (optional)
PARS = {'theta': [1, 6],
        'p_stay': [0, 1],
        'tau': [0, 40, 1.],
        'prelec_gamma': [0, 5],
        'prelec_elevation': [0, 5],
        'pow_gain': [0., np.inf, 1.],
        'w_loss': [0., np.inf, 1.]}

# parameters that are set to fixed values 
FIXED = {'delta': 0.5}

N_ITER = 1
OUTDIR = 'chase_fitresults_frey_individual'

# a list of different parameter combinations that will
# be fit
PARSETS = [['theta', 'p_stay', 'tau'],
           ['theta', 'p_stay', 'tau', 'prelec_gamma', 'prelec_elevation'],
           ['theta', 'p_stay', 'tau', 'pow_gain', 'w_loss'],
           ]

In [10]:
sid = 2

SIM_ID = 'frey_switching_s%s' % sid

FIXED['p_switch'] = emp_p_switch[sid]

for parset in PARSETS:
    
    fitting = {p: PARS[p] for p in parset}

    # initialize the model
    m = CHASEOptionalStoppingSwitchingModel(drift='cpt',
                                            startdist='softmax')

    # fit
    results = fit_mlh(m, problems, data[(data.subject==sid)], 
                      SIM_ID, FIXED, fitting, niter=N_ITER, outdir=OUTDIR)



TypeError: super(type, obj): obj must be an instance or subtype of type